<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/CONDITIONALITY/CPU_CONDITIONAL_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 19:53:17] [setup] RAM Tracking...
[codecarbon INFO @ 19:53:17] [setup] GPU Tracking...
[codecarbon INFO @ 19:53:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:53:17] [setup] CPU Tracking...
[codecarbon WARNING @ 19:53:17] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:53:19] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 19:53:19] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 19:53:19] >>> Tracker's metadata:
[codecarbon INFO @ 19:53:19]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 19:53:19]   Python version: 3.8.5
[codecarbon INFO @ 19:53:19]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 19:53:19]   Available RAM : 27.406 GB
[codecarbon INFO @ 19:53:19]   CPU count: 4
[codecarbon INFO @ 19:53:19]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 19:53:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 17:31:35] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:31:35] Energy consumed for all GPUs : 0.000038 kWh. Total GPU Power : 9.098509586956894 W
[codecarbon INFO @ 17:31:35] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:35] 0.000258 kWh of electricity used since the beginning.
No such comm: dc3b81c614ab453fac033b732cf3d302
[codecarbon INFO @ 17:33:20] Energy consumed for RAM : 0.000342 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 17:33:20] Energy consumed for all GPUs : 0.000301 kWh. Total GPU Power : 9.036904437796405 W
[codecarbon INFO @ 17:33:20] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:33:20] 0.002061 kWh of electricity used since the beginning.
No such comm: 4398c659311d401b92223d4740e1c6ce


In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-22 19:53:10.883861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 19:53:10.999491: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-22 19:53:10.999511: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-22 19:53:11.637429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [ ]:
# Load Dataset from Hugging Face
try:
    dataset = load_dataset("GIZ/policy_classification", data_files="policy_classification.json",use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download("GIZ/policy_classification", filename="policy_classification.json", repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')


Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


In [ ]:
data['check'] = data.apply(lambda x: True if (x['UnconditionalLabel'] == True)
                          or (x['ConditionalLabel'] == True) else False,axis=1)
temp = data[data.check==True].reset_index(drop=True)
print(len(temp))
temp2 = data[data.check ==False].reset_index(drop=True).sample(frac= 0.3)
print(len(temp2))
data = pd.concat([temp,temp2], ignore_index=True)


3416
3177


[codecarbon INFO @ 19:53:34] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:53:34] Energy consumed for all GPUs : 0.000038 kWh. Total GPU Power : 9.231918811308558 W
[codecarbon INFO @ 19:53:34] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:53:34] 0.000259 kWh of electricity used since the beginning.


# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= ['UnconditionalLabel','ConditionalLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "sentence-transformers/all-mpnet-base-v2"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "CONDITIONAL-multilabel-sentencet_w"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.02e-5   #4.04e-5
batch_size = 8
num_train_epochs= 5
weight_decay=0.013
warmup_steps = 200
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
# try:
#     dataset = load_dataset(repo_id, data_files=filename,
#                              use_auth_token=True )
#     data = pd.DataFrame(dataset['train'])
# except:
#     from huggingface_hub import hf_hub_download
#     hf_hub_download(repo_id, filename=filename,
#                     repo_type="dataset",use_auth_token=True,
#     local_dir= 'data/')
#     data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

{0: 'ConditionalLabel', 1: 'UnconditionalLabel'}


[codecarbon INFO @ 19:53:49] Energy consumed for RAM : 0.000086 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:53:49] Energy consumed for all GPUs : 0.000077 kWh. Total GPU Power : 9.261749958395981 W
[codecarbon INFO @ 19:53:49] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:53:49] 0.000517 kWh of electricity used since the beginning.


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 5937
0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312

 Classes Representation in Test Dataset: 656
0 . ConditionalLabel : 228
1 . UnconditionalLabel : 160


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312
Positive Weights 
 {'ConditionalLabel': 1.494712990936556, 'UnconditionalLabel': 2.2625762195121952}
Negative Weights 
 {'ConditionalLabel': 0.7513287775246773, 'UnconditionalLabel': 0.6418378378378379}


[codecarbon INFO @ 19:54:04] Energy consumed for RAM : 0.000128 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:54:04] Energy consumed for all GPUs : 0.000152 kWh. Total GPU Power : 17.947942497818175 W
[codecarbon INFO @ 19:54:04] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:54:04] 0.000812 kWh of electricity used since the beginning.


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

[codecarbon INFO @ 19:54:19] Energy consumed for RAM : 0.000171 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:54:19] Energy consumed for all GPUs : 0.000256 kWh. Total GPU Power : 25.011965522891447 W
[codecarbon INFO @ 19:54:19] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:54:19] 0.001136 kWh of electricity used since the beginning.
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7fa7296a1550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 5937
Validation data: 656


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5937 [00:00<?, ?ex/s]

  0%|          | 0/656 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    push_to_hub = True,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


[codecarbon INFO @ 19:54:34] Energy consumed for RAM : 0.000214 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:54:34] Energy consumed for all GPUs : 0.000361 kWh. Total GPU Power : 25.10373718038511 W
[codecarbon INFO @ 19:54:34] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:54:34] 0.001460 kWh of electricity used since the beginning.


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 19:54:37] [setup] RAM Tracking...
[codecarbon INFO @ 19:54:37] [setup] GPU Tracking...
[codecarbon INFO @ 19:54:37] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:54:38] [setup] CPU Tracking...
[codecarbon WARNING @ 19:54:38] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 19:54:39] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 19:54:39] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 19:54:39] >>> Tracker's metadata:
[codecarbon INFO @ 19:54:39]   Platform system: Linux-5.15.0-1040-azure-x86_64-w

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a MPNetTokenize

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
0,0.772000,0.691434,0.571429,0.288872,0.577184,0.556701,0.301829,0.556701,0.563969,0.289126,0.563864
2,0.417000,0.583623,0.680488,0.382622,0.681317,0.719072,0.394055,0.719072,0.699248,0.383130,0.697911
4,0.163500,0.629603,0.700240,0.412348,0.701639,0.752577,0.414634,0.752577,0.725466,0.409553,0.725686


[codecarbon INFO @ 19:54:49] Energy consumed for RAM : 0.000257 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:54:49] Energy consumed for all GPUs : 0.000511 kWh. Total GPU Power : 36.08324432259792 W
[codecarbon INFO @ 19:54:49] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:54:49] 0.001830 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:55:00] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:55:00] Energy consumed for all GPUs : 0.000285 kWh. Total GPU Power : 68.33069930805195 W
[codecarbon INFO @ 19:55:00] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:55:00] 0.000505 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:55:04] Energy consumed for RAM : 0.000300 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 19:55:04] Energy consumed for all GPUs : 0.000799 kWh. Total GPU Power : 69.14133737441958 W


TrainOutput(global_step=1855, training_loss=0.4439282255352668, metrics={'train_runtime': 2378.1314, 'train_samples_per_second': 12.482, 'train_steps_per_second': 0.78, 'total_flos': 5851326760151040.0, 'train_loss': 0.4439282255352668, 'epoch': 4.99})

In [ ]:
tracker.stop()

[codecarbon INFO @ 20:34:47] Energy consumed for RAM : 0.007100 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:34:47] Energy consumed for all GPUs : 0.043994 kWh. Total GPU Power : 27.91491105368571 W
[codecarbon INFO @ 20:34:47] Energy consumed for all CPUs : 0.029371 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:47] 0.080466 kWh of electricity used since the beginning.


0.029408369732697573

In [ ]:
multi_trainer.push_to_hub()

'https://huggingface.co/ppsingh/CONDITIONAL-multilabel-sentencet_w/tree/main/'

In [ ]:
# class weight
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.772,
  'learning_rate': 3.604640483383686e-05,
  'epoch': 1.0,
  'step': 371},
 {'eval_loss': 0.691433846950531,
  'eval_precision-micro': 0.5714285714285714,
  'eval_precision-samples': 0.2888719512195122,
  'eval_precision-weighted': 0.5771835172883002,
  'eval_recall-micro': 0.5567010309278351,
  'eval_recall-samples': 0.3018292682926829,
  'eval_recall-weighted': 0.5567010309278351,
  'eval_f1-micro': 0.5639686684073106,
  'eval_f1-samples': 0.2891260162601626,
  'eval_f1-weighted': 0.5638642133129377,
  'eval_runtime': 16.2379,
  'eval_samples_per_second': 40.399,
  'eval_steps_per_second': 5.05,
  'epoch': 1.0,
  'step': 371},
 {'loss': 0.6002,
  'learning_rate': 2.7010513595166163e-05,
  'epoch': 2.0,
  'step': 743},
 {'eval_loss': 0.5951880216598511,
  'eval_precision-micro': 0.6421800947867299,
  'eval_precision-samples': 0.3833841463414634,
  'eval_precision-weighted': 0.6447312163045074,
  'eval_recall-micro': 0.6984536082474226,
  'eval_recall-samples': 0.385670

In [ ]:
import json
with open("classifier_info/conditional_sentencet_logs_w.json", "w") as fp:
    json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
# class weights
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ConditionalLabel
[[346  82]
 [ 54 174]] 

UnconditionalLabel
[[453  43]
 [ 42 118]] 



In [ ]:
# class weights
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ConditionalLabel
[[393  73]
 [ 60 168]] 

UnconditionalLabel
[[462  72]
 [ 34 126]] 



In [ ]:
# classw eights
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ConditionalLabel,0.679688,0.763158,0.719008,228.0
1,UnconditionalLabel,0.732919,0.737500,0.735202,160.0
2,micro avg,0.700240,0.752577,0.725466,388.0
3,macro avg,0.706303,0.750329,0.727105,388.0
4,weighted avg,0.701639,0.752577,0.725686,388.0
5,samples avg,0.412348,0.414634,0.409553,388.0


In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.818930,0.843220,0.830898,236.0
1,Buildings,0.647059,0.523810,0.578947,21.0
2,Coastal Zone,0.757143,0.679487,0.716216,78.0
3,Cross-Cutting Area,0.708108,0.642157,0.673522,204.0
4,Disaster Risk Management (DRM),0.662651,0.597826,0.628571,92.0
5,Economy-wide,0.530864,0.430000,0.475138,100.0
6,Education,0.812500,0.481481,0.604651,27.0
7,Energy,0.862179,0.853968,0.858054,315.0
8,Environment,0.652174,0.631579,0.641711,95.0
9,Health,0.838235,0.750000,0.791667,76.0


## Pipeline

In [ ]:
from transformers import pipeline
model_checkpoint = "ppsingh/TAPP-multilabel-climatebert"
pipe = pipeline("text-classification", model=model_checkpoint, top_k=None, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
predictions = pipe(list(test_df['text']))
pred = []
for prediction in predictions:
    pred.append({x['label']:x['score'] for x in prediction})
df_pred = pd.DataFrame(pred)
df_pred['labels'] = df_pred.apply(lambda x: np.array([x[label]
                            for label in label_names]) >= 0.50,axis=1)
y_true = np.array(list(test_df['labels']))
y_pred = np.array(list(df_pred['labels']))

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[417 104]
 [ 69 531]] 

PlansLabel
[[807  77]
 [ 71 166]] 

PolicyLabel
[[939  30]
 [ 28 124]] 

TargetLabel
[[737  62]
 [ 37 285]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.836220,0.885000,0.859919,600.0
1,PlansLabel,0.683128,0.700422,0.691667,237.0
2,PolicyLabel,0.805195,0.815789,0.810458,152.0
3,TargetLabel,0.821326,0.885093,0.852018,322.0
4,micro avg,0.802030,0.843631,0.822305,1311.0
5,macro avg,0.786467,0.821576,0.803515,1311.0
6,weighted avg,0.801289,0.843631,0.821827,1311.0
7,samples avg,0.787021,0.808802,0.782698,1311.0
